In [56]:
from deeplotx import SoftmaxRegression, LongformerEncoder
from util import NUM_CLASSES
lf_encoder = LongformerEncoder(model_name_or_path='severinsimmler/xlm-roberta-longformer-base-16384')

[DEBUG] 2025-08-02 23:46:23,989 deeplotx.embedding : LongformerEncoder initialized on device: cuda.


In [57]:
import torch
from deeplotx.util import sha256
from vortezwohl.cache import LRUCache

CACHE = LRUCache(capacity=16384)

def encode(text: str) -> torch.Tensor:
    key = sha256(text)
    if key in CACHE:
        return CACHE[key]
    emb = lf_encoder.encode(text, cls_only=False).mean(dim=-2, dtype=model.dtype)
    CACHE[key] = emb
    return emb

In [58]:
import os
import pandas as pd
base_path = './data/multilingual_wikineural'
languages = ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru']
train_data = dict()
valid_data = dict()
test_data = dict()
for lang in languages:
    train_data[lang] = pd.read_csv(os.path.join(base_path, f'train_{lang}.csv')).to_dict(orient='records')
    valid_data[lang] = pd.read_csv(os.path.join(base_path, f'val_{lang}.csv')).to_dict(orient='records')
    test_data[lang] = pd.read_csv(os.path.join(base_path, f'test_{lang}.csv')).to_dict(orient='records')

In [59]:
total_train_data = []
total_valid_data = []
total_test_data = []
for lang, data in train_data.items():
    for d in data:
        total_train_data.append((d['tokens'], d['ner_tags']))
for lang, data in valid_data.items():
    for d in data:
        total_valid_data.append((d['tokens'], d['ner_tags']))
for lang, data in test_data.items():
    for d in data:
        total_test_data.append((d['tokens'], d['ner_tags']))

total_train_data[:2], total_valid_data[:2], total_test_data[:2]

([("['Dieses' 'wiederum' 'basierte' 'auf' 'dem' 'gleichnamigen' 'Roman' 'von'\n 'Noël' 'Calef' '.']",
   '[0 0 0 0 0 0 0 0 1 2 0]'),
  ("['Auf' 'Helgoland' 'starben' '2014' 'sieben' 'Jungvögel' 'und' 'fünf'\n 'Altvögel' 'als' 'Verstrickungsopfer' '.']",
   '[0 5 0 0 0 0 0 0 0 0 0 0]')],
 [("['Die' 'Europameisterschaften' 'in' 'Dresden' 'schloss' 'sie' 'mit' 'dem'\n 'zweiten' 'Platz' 'in' 'der' 'Gesamtwertung' 'ab' '.']",
   '[0 0 0 5 0 0 0 0 0 0 0 0 0 0 0]'),
  ('[\'Die\' \'Europameisterschaften\' \'im\' \'selben\' \'Jahr\' \'in\' \'Heerenveen\'\n \'konnte\' \'sie\' \'mit\' \'dem\' \'Gewinn\' \'von\' \'vier\' \'Goldmedaillen\' \'(\'\n \'1000\' \'"\' \'m\' \',\' \'1500\' \'"\' \'m\' \',\' \'3000\' \'"\' \'m\' \',\' \'Gesamtwertung\'\n \')\' \'deutlich\' \'für\' \'sich\' \'entscheiden\' \'.\']',
   '[0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')],
 [("['Er' 'erklärte' ',' 'dass' 'der' 'Allgemeine' 'Nationalkongress'\n 'illegal' 'sei' ',' 'weil' 'das' 'Parlament

In [60]:
train_dataset = {'token': [], 'label': []}
valid_dataset = {'token': [], 'label': []}
test_dataset = {'token': [], 'label': []}

def token_decode(s: str) -> list:
    return [_.strip().replace("'", '').replace('[', '').replace(']', '') for _ in s.replace("' '", '||').replace('\n', '||').split('||')]

def label_decode(s: str) -> list:
    _labels = []
    label_lists = token_decode(s)
    for label_ls in label_lists:
        _labels.extend([int(_) for _ in label_ls.split(' ')])
    return _labels

for tokens, labels in total_train_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for token in tokens:
        train_dataset['token'].append(token)
    for label in labels:
        train_dataset['label'].append(label)
for tokens, labels in total_valid_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for token in tokens:
        valid_dataset['token'].append(token)
    for label in labels:
        valid_dataset['label'].append(label)
for tokens, labels in total_test_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for token in tokens:
        test_dataset['token'].append(token)
    for label in labels:
        test_dataset['label'].append(label)

print(list(zip(train_dataset['token'][:23], train_dataset['label'][:23])))
f'{len(train_dataset["token"])} tokens to train in total.'

[('Dieses', 0), ('wiederum', 0), ('basierte', 0), ('auf', 0), ('dem', 0), ('gleichnamigen', 0), ('Roman', 0), ('von', 0), ('Noël', 1), ('Calef', 2), ('.', 0), ('Auf', 0), ('Helgoland', 5), ('starben', 0), ('2014', 0), ('sieben', 0), ('Jungvögel', 0), ('und', 0), ('fünf', 0), ('Altvögel', 0), ('als', 0), ('Verstrickungsopfer', 0), ('.', 0)]


'17583063 tokens to train in total.'

## 存储嵌入

In [ ]:
train_embeddings = []
valid_embeddings = []
test_embeddings = []
for i, token in enumerate(train_dataset['token']):
    # print(f'\rTrainSet {i}/{len(train_dataset["token"])}')
    train_embeddings.append(encode(token))
print('TrainSet done\n')
for i, token in enumerate(valid_dataset['token']):
    # print(f'\rValidSet {i}/{len(valid_dataset["token"])}')
    valid_embeddings.append(encode(token))
print('ValidSet done\n')
for i, token in enumerate(test_dataset['token']):
    # print(f'\rTestSet {i}/{len(test_dataset["token"])}')
    test_embeddings.append(encode(token))
print('TestSet done\n')

In [ ]:
import pickle

with open('./data/multilingual_wikineural_processed/train.pkl', 'wb') as f:
    _data = {
        'tokens': train_dataset['token'],
        'embeddings': train_embeddings,
        'labels': train_dataset['label']
    }
    pickle.dump(_data, f)
with open('./data/multilingual_wikineural_processed/valid.pkl', 'wb') as f:
    _data = {
        'tokens': valid_dataset['token'],
        'embeddings': valid_embeddings,
        'labels': valid_dataset['label']
    }
    pickle.dump(_data, f)
with open('./data/multilingual_wikineural_processed/test.pkl', 'wb') as f:
    _data = {
        'tokens': test_dataset['token'],
        'embeddings': test_embeddings,
        'labels': test_dataset['label']
    }
    pickle.dump(_data, f)

## 训练

In [61]:
model = SoftmaxRegression(input_dim=768, output_dim=NUM_CLASSES, num_heads=4, num_layers=3, expansion_factor=1.25, bias=True, dropout_rate=0.2, head_layers=2)
print(model)

Model_Name: SoftmaxRegression
In_Features: 768
Out_Features: 9
Device: cuda
Dtype: torch.float32
Total_Parameters: 49642017
Trainable_Parameters: 49642017
NonTrainable_Parameters: 0
-------------------------------
SoftmaxRegression(
  (multi_head_ffn_layers): ModuleList(
    (0-2): 3 x MultiHeadFeedForward(
      (expand_proj): Linear(in_features=768, out_features=3072, bias=True)
      (ffn_heads): ModuleList(
        (0-3): 4 x FeedForward(
          (ffn_layers): ModuleList(
            (0-1): 2 x FeedForwardUnit(
              (up_proj): Linear(in_features=768, out_features=960, bias=True)
              (down_proj): Linear(in_features=960, out_features=768, bias=True)
              (parametric_relu): PReLU(num_parameters=1)
              (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
            )
          )
        )
      )
      (out_proj): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (out_proj): Linear(in_features=768, out_features=9, 

In [62]:
# train
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter

train_step = 0
valid_step = 0
writer = SummaryWriter()

acc_train_loss = 0.
acc_valid_loss = 0.
eval_interval = 2000
log_interval = 200

In [ ]:
from random import randint
from util import one_hot

elastic_net_param = {
    'alpha': 2e-4,
    'rho': 0.2
}
learning_rate = 2e-5
num_epochs = 1500
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    model.train()
    for i, _token in enumerate(train_dataset['token']):
        _label = train_dataset['label'][i]
        _one_hot_label = one_hot(_label).to(model.dtype).to(model.device)
        outputs = model.forward(encode(_token))
        loss = loss_function(outputs, _one_hot_label) + model.elastic_net(alpha=elastic_net_param['alpha'], rho=elastic_net_param['rho'])
        acc_train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if train_step % log_interval == 0 and train_step > 0:
            writer.add_scalar('train/loss', acc_train_loss / log_interval, train_step)
            print(f'- Train Step {train_step} Loss {acc_train_loss / log_interval} \ \nPred={outputs.tolist()} \nLabel={_one_hot_label.tolist()}', flush=True)
            acc_train_loss = 0.
        train_step += 1
        if train_step % eval_interval == 0:
            model.eval()
            rand_idx = randint(0, len(valid_dataset['token']) - 1001)
            with torch.no_grad():
                for _i, __token in enumerate(valid_dataset['token'][rand_idx: rand_idx + 1000]):
                    _label = valid_dataset['label'][_i]
                    _one_hot_label = one_hot(_label).to(model.dtype).to(model.device)
                    outputs = model.forward(encode(__token))
                    loss = loss_function(outputs, _one_hot_label)
                    acc_valid_loss += loss.item()
                    if valid_step % log_interval == 0 and valid_step > 0:
                        writer.add_scalar('valid/loss', acc_valid_loss / log_interval, valid_step)
                        print(f'- Valid Step {valid_step} Loss {acc_valid_loss / log_interval} \ \nPred={outputs.tolist()} \nLabel={_one_hot_label.tolist()}', flush=True)
                        acc_valid_loss = 0.
                    valid_step += 1
            model.train()

- Train Step 200 Loss 34.89888299942017 \ 
Pred=[0.9995613694190979, 5.428781150840223e-05, 4.993554102838971e-05, 5.346506804926321e-05, 5.458595842355862e-05, 6.005772593198344e-05, 5.2782044804189354e-05, 7.049780833767727e-05, 4.299043212085962e-05] 
Label=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- Train Step 400 Loss 29.45360107421875 \ 
Pred=[0.9994613528251648, 6.147123349364847e-05, 6.324265996227041e-05, 7.069380080793053e-05, 6.162947829579934e-05, 7.940407522255555e-05, 6.487691280199215e-05, 7.839842874091119e-05, 5.8994366554543376e-05] 
Label=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- Train Step 600 Loss 24.354979104995728 \ 
Pred=[0.9994102716445923, 6.913138349773362e-05, 6.816555105615407e-05, 7.373194966930896e-05, 7.238450780278072e-05, 8.682082261657342e-05, 7.362740143435076e-05, 8.081114356173202e-05, 6.499783194158226e-05] 
Label=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- Train Step 800 Loss 19.590502614974977 \ 
Pred=[0.9994149208068848, 7.2720213

In [65]:
model.eval()
test_tokens = ['Mike', 'John', 'Smith', 'London', 'NYC', 'HongKong', 'China', 'South Africa', 'Korea']
with torch.no_grad():
    for _tok in test_tokens:
        _dist = model.forward(encode(_tok))
        print(f'Token={_tok}, Class={torch.argmax(_dist)}', flush=True)
model.train()
'Test finished.'

Token=Mike, Class=0
Token=John, Class=0
Token=Smith, Class=0
Token=London, Class=0
Token=NYC, Class=0
Token=HongKong, Class=0
Token=China, Class=0
Token=South Africa, Class=0
Token=Korea, Class=0


'Test finished.'